# Natural Language Processing 2

## 문서 읽어와서 단어 및 빈도수 구하기

In [1]:
import nltk
from konlpy.tag import Kkma
import glob
import re

fileList = glob.glob("D:/Study/BigData/JupyterNotebook/Study/doc/*")
# 해당 path에 대해서 파일의 path를 가져온다.

In [3]:
docNouns = {} #
maxCount = {} # 문서별로 명사의 빈도수 중 가장 높은 카운트를 지닌다.
contentAll = ""

for file in fileList:
    docid = file[-17:] # 문서명
    
    with open(file, "r", encoding="utf-8") as fp:
        maxCount[docid] = 0 # 최대 빈도수 초기화
        content = fp.read() # 파일을 읽는다.
        content = re.sub(r"[\s]{2,}", "", content) # 공백이 2개 이상이면 뺀다.

    nouns = {} #하나의 문서에 대한 명사들
    for sentence in nltk.sent_tokenize(content): # 문서의 한 문장씩 for문
        for word in nltk.word_tokenize(sentence): # 한 문장을 whitespace, 특수문자 기준으로 tokenize된 단어 for문
            for w in Kkma().nouns(word): # 명사들에 대해서 for문
                if w in nouns.keys(): # 해당 단어가 이미 nouns dict에 있을 때 
                    nouns[w] += 1 # 카운트를 1 더해준다.
                else: # 해당 단어가 dict에 없을 경우
                    nouns[w] = 1 # 카운트를 1로 할당.
                if maxCount[docid] < nouns[w]: # 명사의 빈도수가 현재 maxcount에 저장되어 있는 값보다 크다면
                    maxCount[docid] = nouns[w] # 명사 빈도수의 큰 값을 갱신해줌
                    
    contentAll += "\n" + content # 해당 파일의 내용을 이어서 저장
    docNouns[docid] = nouns 
    # key값은 문서명, 해당 문서에 존재하는 단어와 빈도수를 저장한 dict가 value 저장

In [4]:
for k,v in docNouns.items():
    print("{0}  -  {1} / {2}".format(k, len(v), maxCount[k])) 
    # 문서명, 각 문서에 대해서 단어의 수, 단어중 최대 빈도수

20181028161200956  -  294 / 20
20181028161236972  -  238 / 15
20181028162436248  -  183 / 10
20181028163000336  -  633 / 32
20181028163145387  -  338 / 11
20181028163253411  -  260 / 18
20181028163257414  -  183 / 9
20181028163851574  -  211 / 14
20181028164102632  -  201 / 12
20181028164104633  -  235 / 12
20181028164604773  -  322 / 14
20181028164903850  -  311 / 16


## 역색인 (Inverted Indexing)

낱말이나 숫자와 같은 내용물로부터의 매핑 정보를 데이터베이스 파일의 특정 지점이나 문서 또는 문서 집합 안에 저장하는 indexing data structure이다.

In [6]:
uniqueNouns = [] 

for sentence in nltk.sent_tokenize(contentAll): # 텍스트를 문장으로 쪼개서 for문
    nouns = []
    for word in nltk.word_tokenize(sentence): # 문장을 단어로 쪼개서 for문
        nouns.extend(Kkma().nouns(word)) # 명사만 골라서 nouns list에 넣는다. 
        # 검색 엔진에서도 명사로서 간단히 검색할 때 검색 성능이 제일 좋다. 
        nouns = list(set(nouns)) #unique한 명사 단어들만 가져온다.(중복 제거)
    uniqueNouns.extend(nouns) #리스트에 넣는다.
    uniqueNouns = list(set(uniqueNouns)) # 중복 단어 제거
print(len(uniqueNouns)) # 명사들 길이

2425


In [7]:
invertedIndex = {}

for noun in uniqueNouns: # 명사들에 대해서 for문
    invertedIndex[noun] = [] 
    
    for k,v in docNouns.items(): # key값은 문서명, value값은 {단어:빈도수} dict이다.
        if noun in v: # 해당 dict의 키값인 단어와 일치한다면
            invertedIndex[noun].append(k) 
            # 해당 단어 key값에 대해 단어가 속한 문서를 value값으로 준다.

In [6]:
print(invertedIndex) #각 단어들에 대해서 단어가 속한 문서를 알 수 있다.

{'법제사법위원회': ['20180801140802122'], '디자이너': ['20180801140951174'], '신경': ['20180801140352961'], '압수수색': ['20180801143420047'], '처리': ['20180801141549383'], '이': ['20180801140352961', '20180801140802122', '20180801140921160', '20180801140951174', '20180801141549383', '20180801141712421', '20180801142859853', '20180801142952883', '20180801143420047'], '추천': ['20180801140802122', '20180801141712421'], '북': ['20180801140352961', '20180801140802122', '20180801140921160'], '현대적': ['20180801141138247'], '합류': ['20180801143420047'], '포착': ['20180801140352961'], '후': ['20180801140352961', '20180801140951174', '20180801141549383'], '2시': ['20180801141138247'], '이완영': ['20180801140802122'], '드라이브': ['20180801142952883'], '검토': ['20180801140802122', '20180801142952883', '20180801143420047'], '네덜란드': ['20180801141549383'], '반대': ['20180801140951174'], '글': ['20180801142859853'], '세월호팀': ['20180801143420047'], '촬영': ['20180801140921160'], '규': ['20180801140622059'], '기무요원': ['20180801143420047'], '관측

## TF-IDF

TF-IDF는 단어 빈도와 역문서 빈도의 곱이다. 두 값을 산출하는 방식에는 여러 가지가 있다.  

__단어 빈도(Term Frequency)__   
$tf(t,d)$의 경우, 이 값을 산출하는 가장 간단한 방법은 단순히 문서 내에 나타나는 해당 단어의 총 빈도수를 사용하는 것이다.  
문서 d 내에서 단어 t의 총 빈도를 $f(t,d)$라 할 경우, 가장 단순한 tf 산출 방식은 $tf(t,d) = f(t,d)$로 표현된다.   
그 밖에 TF값을 산출하는 방식에는 다음과 같은 것들이 있다.  

* 불린 빈도: $tf(t,d) = t$가 $d$에 한 번이라도 나타나면 1, 아니면 0;
* 로그 스케일 빈도: $tf(t,d) = log (f(t,d) + 1)$;
* 증가 빈도: 문서의 길이에 따라 단어의 빈도값 조정  
  
$$\mathrm{tf}(t,d) = 0.5 + \frac{0.5 \times \mathrm{f}(t, d)}{\max\{\mathrm{f}(w, d):w \in d\}}$$

In [9]:
#TF

kRation = 0.5
TF = {}

for k,v in docNouns.items(): # docNouns에 대한 for문
    tfList = {}
    for w in v: # 문서의 단어장 dict에 대한 for문 -> w에는 key값인 단어가 들어갈 것임.
        tfList[w] = kRation + (1 - kRation) * (v[w] / maxCount[k]) # tf값을 구한다.
        # double normalization
        print("{0} | {1} + {2} * ({3} / {4}) = {5} ".format(
        w, kRation, (1-kRation), v[w], maxCount[k], tfList[w]))

        TF[k] = tfList
        # 각 문서명을 key로 가지며, {단어 : TF값} 의 dict를 value로 가짐.

미래 | 0.5 + 0.5 * (1 / 20) = 0.525 
미래당 | 0.5 + 0.5 * (1 / 20) = 0.525 
당 | 0.5 + 0.5 * (1 / 20) = 0.525 
등 | 0.5 + 0.5 * (12 / 20) = 0.8 
고발 | 0.5 + 0.5 * (2 / 20) = 0.55 
사건 | 0.5 + 0.5 * (5 / 20) = 0.625 
피고 | 0.5 + 0.5 * (1 / 20) = 0.525 
피고발인 | 0.5 + 0.5 * (1 / 20) = 0.525 
발인 | 0.5 + 0.5 * (1 / 20) = 0.525 
신분 | 0.5 + 0.5 * (1 / 20) = 0.525 
경찰 | 0.5 + 0.5 * (9 / 20) = 0.725 
경찰조사 | 0.5 + 0.5 * (3 / 20) = 0.575 
조사 | 0.5 + 0.5 * (13 / 20) = 0.825 
하루 | 0.5 + 0.5 * (3 / 20) = 0.575 
이재명 | 0.5 + 0.5 * (4 / 20) = 0.6 
유명 | 0.5 + 0.5 * (1 / 20) = 0.525 
로 | 0.5 + 0.5 * (1 / 20) = 0.525 
로펌인 | 0.5 + 0.5 * (1 / 20) = 0.525 
펌인 | 0.5 + 0.5 * (1 / 20) = 0.525 
법무 | 0.5 + 0.5 * (1 / 20) = 0.525 
법무법인 | 0.5 + 0.5 * (1 / 20) = 0.525 
법인 | 0.5 + 0.5 * (1 / 20) = 0.525 
변호사 | 0.5 + 0.5 * (13 / 20) = 0.825 
선임 | 0.5 + 0.5 * (4 / 20) = 0.6 
법리 | 0.5 + 0.5 * (7 / 20) = 0.675 
검토 | 0.5 + 0.5 * (4 / 20) = 0.6 
준비 | 0.5 + 0.5 * (5 / 20) = 0.625 
확인 | 0.5 + 0.5 * (2 / 20) = 0.55 
이 | 0.5 + 0.5 * (20 

사무 | 0.5 + 0.5 * (2 / 32) = 0.53125 
사무행정 | 0.5 + 0.5 * (1 / 32) = 0.515625 
주요 | 0.5 + 0.5 * (1 / 32) = 0.515625 
정책 | 0.5 + 0.5 * (1 / 32) = 0.515625 
며 | 0.5 + 0.5 * (1 / 32) = 0.515625 
국회 | 0.5 + 0.5 * (3 / 32) = 0.546875 
개정 | 0.5 + 0.5 * (1 / 32) = 0.515625 
요청 | 0.5 + 0.5 * (1 / 32) = 0.515625 
자체 | 0.5 + 0.5 * (1 / 32) = 0.515625 
무력화 | 0.5 + 0.5 * (1 / 32) = 0.515625 
수준 | 0.5 + 0.5 * (2 / 32) = 0.53125 
반대 | 0.5 + 0.5 * (6 / 32) = 0.59375 
세력 | 0.5 + 0.5 * (2 / 32) = 0.53125 
자기 | 0.5 + 0.5 * (1 / 32) = 0.515625 
모순 | 0.5 + 0.5 * (1 / 32) = 0.515625 
추천위원회 | 0.5 + 0.5 * (1 / 32) = 0.515625 
위원회 | 0.5 + 0.5 * (1 / 32) = 0.515625 
위원 | 0.5 + 0.5 * (1 / 32) = 0.515625 
위촉 | 0.5 + 0.5 * (1 / 32) = 0.515625 
특검 | 0.5 + 0.5 * (10 / 32) = 0.65625 
건 | 0.5 + 0.5 * (2 / 32) = 0.53125 
배당 | 0.5 + 0.5 * (8 / 32) = 0.625 
다 | 0.5 + 0.5 * (1 / 32) = 0.515625 
무작위 | 0.5 + 0.5 * (1 / 32) = 0.515625 
명령 | 0.5 + 0.5 * (1 / 32) = 0.515625 
예규 | 0.5 + 0.5 * (5 / 32) = 0.578125 
사무분담 | 0.5 + 0.

24세 | 0.5 + 0.5 * (1 / 16) = 0.53125 
세 | 0.5 + 0.5 * (1 / 16) = 0.53125 
중 | 0.5 + 0.5 * (1 / 16) = 0.53125 
3 | 0.5 + 0.5 * (1 / 16) = 0.53125 
3분 | 0.5 + 0.5 * (1 / 16) = 0.53125 
분 | 0.5 + 0.5 * (1 / 16) = 0.53125 
건강문제 | 0.5 + 0.5 * (1 / 16) = 0.53125 
문제 | 0.5 + 0.5 * (1 / 16) = 0.53125 
마약 | 0.5 + 0.5 * (1 / 16) = 0.53125 
마약복용 | 0.5 + 0.5 * (1 / 16) = 0.53125 
복용 | 0.5 + 0.5 * (1 / 16) = 0.53125 
병력자원 | 0.5 + 0.5 * (1 / 16) = 0.53125 
부적합 | 0.5 + 0.5 * (1 / 16) = 0.53125 
상황 | 0.5 + 0.5 * (2 / 16) = 0.5625 
어려움 | 0.5 + 0.5 * (1 / 16) = 0.53125 
가중 | 0.5 + 0.5 * (1 / 16) = 0.53125 
설명 | 0.5 + 0.5 * (1 / 16) = 0.53125 
증원 | 0.5 + 0.5 * (2 / 16) = 0.5625 
도 | 0.5 + 0.5 * (1 / 16) = 0.53125 
도널 | 0.5 + 0.5 * (1 / 16) = 0.53125 
널 | 0.5 + 0.5 * (1 / 16) = 0.53125 
트럼프 | 0.5 + 0.5 * (4 / 16) = 0.625 
행정부 | 0.5 + 0.5 * (3 / 16) = 0.59375 
공약 | 0.5 + 0.5 * (1 / 16) = 0.53125 
기도 | 0.5 + 0.5 * (1 / 16) = 0.53125 
군 | 0.5 + 0.5 * (6 / 16) = 0.6875 
감축 | 0.5 + 0.5 * (2 / 16) = 0.5625 
기조 

__역문서 빈도(Inverse Document Frequency)__  
는 한 단어가 문서 집합 전체에서 얼마나 공통적으로 나타나는지를 나타내는 값이다.  
전체 문서의 수를 해당 단어를 포함한 문서의 수로 나눈 뒤 로그를 취하여 얻을 수 있다.

* $ |D| $: 문서 집합 D의 크기, 또는 전체 문서의 수
* $|\{d \in D: t \in d\}| $ : 단어 $t$가 포함된 문서의 수.(즉, $ \mathrm{tf}(t,d) \neq 0$). 단어가 전체 말뭉치 안에 존재하지 않을 경우 이는 분모가 0이 되는 결과를 가져온다. 이를 방지하기 위해 $1 + |\{d \in D: t \in d\}|$로 쓰는 것이 일반적이다.  

$$\mathrm{idf}(t, D) =  \log \frac{|D|}{|\{d \in D: t \in d\}|}$$






In [11]:
#IDF
from math import log10

In [14]:
docSize = len(fileList) # 문서의 개수
TFIDF = {}

"""
TF = doc1 | 단어1: 0.54, 단어2: 1.0...
"""

print(TF.items()) # 각 문서에 대한 단어와 그 단어의 TF값.

for k,v in TF.items(): # 각 문서에 대한 단어와 그 단어에 대한 TF값
    idfList = {}
    for t in v: # 단어장 내의 단어 for문
        idf = log10(docSize / len(invertedIndex[t])) # idf값을 구한다.
        idfList[t] = v[t] * idf # tf값 x idf값 = TF-IDF 값
#        print("{0} | {1} * log({2} / {3}) =  {4}".format(
#            t, v[t], docSize, len(invertedIndex[t]), idfList[t]
#        ))
    
    TFIDF[k] = idfList 
    # 해당 문서 key에 대한 {단어 : tf-idf값} dict를 value로 지닌다.

dict_items([('20181028161200956', {'미래': 0.525, '미래당': 0.525, '당': 0.525, '등': 0.8, '고발': 0.55, '사건': 0.625, '피고': 0.525, '피고발인': 0.525, '발인': 0.525, '신분': 0.525, '경찰': 0.725, '경찰조사': 0.575, '조사': 0.825, '하루': 0.575, '이재명': 0.6, '유명': 0.525, '로': 0.525, '로펌인': 0.525, '펌인': 0.525, '법무': 0.525, '법무법인': 0.525, '법인': 0.525, '변호사': 0.825, '선임': 0.6, '법리': 0.675, '검토': 0.6, '준비': 0.625, '확인': 0.55, '이': 1.0, '지사': 0.875, '진행': 0.525, '29': 0.55, '29일': 0.55, '일': 0.575, '휴가': 0.525, '상황': 0.55, '이날': 0.55, '경기': 0.7, '경기도': 0.6, '경기도청': 0.525, '도': 0.6, '청': 0.525, '출근': 0.525, '자택': 0.525, '분당': 0.625, '분당경찰서': 0.575, '경찰서': 0.625, '예정': 0.575, '경기도지사': 0.55, '사진': 0.55, '이한': 0.525, '이한형': 0.525, '형': 0.525, '기자': 0.575, '이지사': 0.525, '측': 0.55, '법리검토': 0.575, '조사준비': 0.55, '문제': 0.525, '변호인': 0.525, '화': 0.525, '화우': 0.525, '우': 0.525, '2003': 0.525, '2003년': 0.525, '년': 0.55, '설립': 0.525, '2007': 0.525, '2011': 0.525, '2011년': 0.525, '퇴직': 0.525, '검사': 0.525, '166': 0.525, '166명': 0.525,

TF-IDF는 다음과 같이 표현된다.
$$\mathrm{tf}(t,d) = 0.5 + \frac{0.5 \times \mathrm{f}(t, d)}{\max\{\mathrm{f}(w, d):w \in d\}}$$  
  
$$\mathrm{idf}(t, D) =  \log \frac{|D|}{|\{d \in D: t \in d\}|}$$
### $$\mathrm{tfidf}(t,d,D) = \mathrm{tf}(t,d) \times \mathrm{idf}(t, D)$$

## Cosine-Similarity

__코사인 유사도(cosine similarity)__는 내적공간의 두 벡터간 각도의 코사인값을 이용하여 측정된 벡터간의 유사한 정도를 의미한다.  
각도가 0°일 때의 코사인값은 1이며, 다른 모든 각도의 코사인값은 1보다 작다.  
따라서 이 값은 <U>벡터의 크기가 아닌 방향의 유사도를 판단하는 목적으로 사용</U>되며,  
두 벡터의 방향이 완전히 같을 경우 1, 90°의 각을 이룰 경우 0,  
180°로 완전히 반대 방향인 경우 -1의 값을 갖는다.  
이 때 벡터의 크기는 값에 아무런 영향을 미치지 않는다.  
코사인 유사도는 특히 결과값이 [0,1]의 범위로 떨어지는 양수 공간에서 사용된다.

두 벡터의 코사인값은 유클리디안 스칼라곱 공식에서 유도할 수 있다.  

#### $$\mathbf{a}\cdot\mathbf{b}
=\left\|\mathbf{a}\right\|\left\|\mathbf{b}\right\|\cos\theta$$

속성 $A$, $B$의 벡터값이 각각 주어졌을 때, 코사인 유사도 $cos(θ)$는 벡터의 스칼라곱과 크기로 다음과 같이 표현할 수 있다.

### $$\text{similarity} = \cos(\theta) = {A \cdot B \over \|A\| \|B\|} = \frac{ \sum\limits_{i=1}^{n}{A_i \times B_i} }{ \sqrt{\sum\limits_{i=1}^{n}{(A_i)^2}} \times \sqrt{\sum\limits_{i=1}^{n}{(B_i)^2}} }$$

이렇게 계산된 유사도는 &minus;1에서 1까지의 값을 가지며, &minus;1은 서로 완전히 반대되는 경우, 0은 서로 독립적인 경우, 1은 서로 완전히 같은 경우를 의미한다.  
  
텍스트 매칭에 적용될 경우, $A$, $B$의 벡터로는 일반적으로 해당 문서에서의 단어 빈도가 사용된다. 코사인 유사도는 문서들간의 유사도를 비교할 때 문서의 길이를 정규화하는 방법의 하나라고 볼 수도 있다.  
  
정보 검색의 경우, 문서의 단어 빈도(tf-idf 가중치)가 음의 값이 되는 것이 불가능하므로 두 문서 사이의 코사인 유사도는 0에서 1까지의 값으로 표현되며, 두 단어 빈도간의 각도는 90°를 넘을 수 없다.  

In [16]:
query = "미국과 중국 중 미국 이겨라!"

queryNouns = {}
maxCount = 0

for word in nltk.word_tokenize(query): 
    # 한 문장을 whitespace, 특수문자 기준으로 tokenize된 단어 for문
    for w in Kkma().nouns(word): 
        # 명사들에 대해서 for문
        if w in queryNouns.keys(): # 만약 해당 명사가 이미 dict내에 있다면 
            queryNouns[w] += 1 # 카운트를 1 증가
        else: # 해당 명사가 dict내에 없다면
            queryNouns[w] = 1 # 카운트를 1로 준다.
        if maxCount < queryNouns[w]: # 명사의 빈도수가 현재 maxcount에 저장되어 있는 값보다 크다면
            maxCount = queryNouns[w] # 명사 빈도수의 큰 값을 갱신해줌
    
print(len(queryNouns), maxCount) # 단어의 개수, 단어의 최대 빈도수
print(queryNouns) # {단어 : 빈도수}의 dict

4 2
{'미국': 2, '중국': 1, '중': 1, '라': 1}


In [17]:
queryWeight = {}

for k,v in queryNouns.items() : # k는 단어, v는 빈도수
    queryWeight[k] = kRation + (1-kRation) * (v / maxCount) # tf값
    print("{0} | {1} + {2} * ({3} / {4}) = {5}".format(
    k, kRation, (1-kRation), v, maxCount, queryWeight[k]
    )) # tf 식과 값 출력
    

미국 | 0.5 + 0.5 * (2 / 2) = 1.0
중국 | 0.5 + 0.5 * (1 / 2) = 0.75
중 | 0.5 + 0.5 * (1 / 2) = 0.75
라 | 0.5 + 0.5 * (1 / 2) = 0.75


In [18]:
from math import sqrt

docLength = {} #문서1: 길이, 문서2: 길이... 이런식으로 들어갈 것임.

for k, v in TFIDF.items(): # {문서 : {단어 : TF-IDF값}}
    sumPow = 0.0
    
    for t in v: # {단어 : TF-IDF값}에서의 단어 for문
        sumPow += v[t]**2 # 단어의 TF-IDF값을 제곱해서 더한다. 
    
    docLength[k] = sqrt(sumPow) # 그리고 루트 씌운 값

#     k #문서ID
#     v #단어들의 TF-IDF값들을 제곱해서 더하고 루트 씌운 값

In [22]:
result = {}

for k,v in queryWeight.items(): # {단어 : TF값} 
    if k in invertedIndex.keys(): # invertedIndex = { 단어 : [문서1, 문서2,..]} 
        # 역색인에 있는 단어에 포함되어있는 경우 
        for docid in invertedIndex[k]: # 해당 단어가 포함된 문서들 for문
            if docid in result.keys(): # 만약 result dict에 단어가 있다면
                result[docid] += TFIDF[docid][k] * queryWeight[k] 
                # TF-IDF값에 TF값 곱한 것을 더함
            else: # result dict에 해당 단어가 없을 때
                result[docid] = TFIDF[docid][k] * queryWeight[k] 
                # 해당 단어의 TF-IDF값에 TF값 곱한다.
                
#             print(" - DocID:{0}, Weight:{1}".format(docid, TFIDF[docid][k]))
            
for k, v in result.items(): # {문서 : 위에서 곱해서 더한 값들}
    result[k] = v / docLength[k] #  코사인 유사도를 구해서 저장

result = dict(sorted(result.items(), key=lambda k:k[1], reverse=True))

for k, v in result.items():
    print("[{0}] 유사도 : {1}".format(k,v))
    with open("D:/Study/BigData/JupyterNotebook/Study/doc/"+k, "r", encoding='utf-8') as fp:
        print(fp.readlines(10))
    # 해당 문서와 유사도를 출력해주고, 파일을 10줄 읽어서 출력

[20181028164604773] 유사도 : 0.13835882149866327
['\n', '\n', '[한겨레] \n', '\n', '\n']
[20181028163145387] 유사도 : 0.09384020313615025
['\n', '\n', '\n', '\n', '\n', '           김동연 경제부총리 겸 기획재정부 장관(왼쪽 두 번째)이 23일 오전 서울 세종로 정부서울청사에서 열린 대외경제장관회의에서 모두발언을 하고 있다. 2018.10.23/뉴스1 <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지>\n']
[20181028164903850] 유사도 : 0.049567244764073544
['\n', '\n', '\n', '\n', '\n', '           세계 1,2차 대전 당시 미 육군의 모병 안내 포스터에 사용된 유명한 엉클샘을 패러디한 모습. 게티 이미지 뱅크\n']
[20181028164104633] 유사도 : 0.04020027454494461
['\n', '\n', '\n', '\n', '\n', '           【리우데자네이루(브라질)=AP/뉴시스】브라질 사회자유당의 자이르 보우소나루 의원이 지난 7월 22일(현지시간) 리우데자네이루에서 열린 당대회에서 대선 후보 지명을 수락한 후 부인 미셸(오른쪽) 및 변호사 자나이나 파쇼알과 함께 브라질 국가를 부르고 있다. 수감 중인 루이스 이나시오 룰라 다 시우바에 이어 여론조사에서 2위를 기록하고 있는 극우 성향의 군출신 보우소나루는 대선 출마 선언으로 단숨에 유력 후보로 떠올랐다. 2018.7.23 <저작권자ⓒ 공감언론 뉴시스통신사. 무단전재-재배포 금지.>\n']
[20181028163000336] 유사도 : 0.03229229071778495
['\n', '\n', '\n', '\n', '\n', '"법원은 검찰보다 영리했기에 알아서 개혁을 했어요"\n']
[20181028161236972] 유사도 : 0.012144665325456003
['\n', '\

통계기반으로 만든 간단한 검색엔진이다.